# Combine straw info into a couple of csvs 
Initial processing of two types of straw info.

Two sources of info:
1. leak rate raw data files: tens of thousands of txt files -- more than one for each straw
2. database, which contains batch, ppg, and panel info for most straws since 173

Do a little processing of (1) here: re-extract the leak rates and combine them into a df.

Save to CSVs so they can be imported from another sheet for analysis.

In [1]:
import pandas as pd
from pathlib import Path
import sys
sys.path.append('/Users/Ben/Projects/mu2e-tracker-factory/')

## 1. Leak Rate of every straw
- Loop all 40,000 raw data files and fit the data to get a leak rate
- Takes ~minutes
- End result: a data frame with columns: straw, raw data file, fit value, fit error, fit status
- Minimal cleaning: remove UNKNOWN, FAILED fits, and unphysical leaks < 0.

In [2]:
# indir, location of raw data
#from guis.common.getresources import GetProjectPaths
#indir = GetProjectPaths()["strawleakdata"] / "raw_data"
indir = Path("/Users/Ben/Projects/mu2e-tracker-factory/data/all_straw_leak_data") # collected ALL the data
print(indir)
print(indir.is_dir())

/Users/Ben/Projects/mu2e-tracker-factory/data/all_straw_leak_data
True


In [3]:
# fitting function
from calc_straw_leak import refit, NotEnoughData

In [ ]:
# one row for each file – many rows for one straw are allowed.
leak_data = []
for raw_data_file in indir.rglob("*.txt"):
    straw = str(raw_data_file.name)[2:7]
    
    #if straw != "23842":
    #    continue
        
    try:
        straw = int(straw)
    except ValueError:
        print("BAD FILE (FILENAME)", raw_data_file.name)
        continue

    try:
        leak_rate, leak_rate_err, leak_status, chamber = refit(raw_data_file)
    except NotEnoughData:
        continue
    except IndexError as e:
        print("BAD FILE (INDEX)", e, raw_data_file.name)
    except ValueError as e:
        print("BAD FILE (VALUE)", raw_data_file.name)
        #print(e)
        continue
        
    leak_data.append(
        {
            'straw':straw,
            'leak_data_file':str(raw_data_file.name),
            'rate': leak_rate,
            'rate_err':leak_rate_err,
            #'status':leak_status.value,
            'status':leak_status,
            'chamber':chamber,
        }
    )

In [25]:
leak_df = pd.DataFrame(leak_data)
leak_df['status'] = leak_df['status'].apply(lambda x: x.value)
leak_df

,straw,leak_data_file,rate,rate_err,status,chamber
0,11163,ST11163_chamber1_2019_08_29_rawdata.txt,0.030628,0.001737+0.000000j,U,1
1,11473,ST11473_chamber0_2019_09_04_rawdata.txt,0.000042,0.000009+0.000000j,P,0
2,10869,ST10869_chamber49_2019_08_26_rawdata.txt,0.000030,0.000007+0.000000j,P,49
3,11830,ST11830_chamber24_2019_09_12_rawdata.txt,0.000063,0.000010+0.000000j,U,24
4,11874,ST11874_chamber22_2019_09_13_rawdata.txt,0.000075,0.000007+0.000000j,P,22
...,...,...,...,...,...,...
32405,14992,ST14992_chamber11_2021_04_20_rawdata.txt,0.001840,0.000068+0.000000j,F,11
32406,14319,ST14319_chamber15_2021_04_28_rawdata.txt,0.000056,0.000001+0.000000j,P,15
32407,14495,ST14495_chamber4_2022_06_01_rawdata.txt,0.000062,0.000003+0.000000j,P,4
32408,14032,ST14032_chamber1_2020_03_06_rawdata.txt,0.000059,0.000009+0.000000j,P,1


In [29]:
# save all rates to csv for chamber analysis
leak_df.drop(leak_df[ leak_df['status'] == "U"].index , inplace=True)
leak_df.to_csv("all_leak_tests_2022-06.csv")
leak_df

,straw,leak_data_file,rate,rate_err,status,chamber
1,11473,ST11473_chamber0_2019_09_04_rawdata.txt,0.000042,0.000009+0.000000j,P,0
2,10869,ST10869_chamber49_2019_08_26_rawdata.txt,0.000030,0.000007+0.000000j,P,49
4,11874,ST11874_chamber22_2019_09_13_rawdata.txt,0.000075,0.000007+0.000000j,P,22
6,10547,ST10547_chamber37_2019_08_22_rawdata.txt,0.000077,0.000006+0.000000j,P,37
7,10268,ST10268_chamber7_2019_08_20_rawdata.txt,0.000093,0.000005+0.000000j,P,7
...,...,...,...,...,...,...
32405,14992,ST14992_chamber11_2021_04_20_rawdata.txt,0.001840,0.000068+0.000000j,F,11
32406,14319,ST14319_chamber15_2021_04_28_rawdata.txt,0.000056,0.000001+0.000000j,P,15
32407,14495,ST14495_chamber4_2022_06_01_rawdata.txt,0.000062,0.000003+0.000000j,P,4
32408,14032,ST14032_chamber1_2020_03_06_rawdata.txt,0.000059,0.000009+0.000000j,P,1


In [23]:
# remove lines with negative leak values
leak_df.drop(leak_df[ leak_df['rate'] <= 0 ].index , inplace=True)

#leak_df = leak_df.rename_axis(None)

# remove failed, unknown tests
leak_df.drop(leak_df[ leak_df['status'] == "F"].index , inplace=True)
leak_df.drop(leak_df[ leak_df['status'] == "U"].index , inplace=True)
leak_df

,straw,leak_data_file,rate,rate_err,status,chamber
1,11473,ST11473_chamber0_2019_09_04_rawdata.txt,0.000042,0.000009+0.000000j,P,0
2,10869,ST10869_chamber49_2019_08_26_rawdata.txt,0.000030,0.000007+0.000000j,P,49
4,11874,ST11874_chamber22_2019_09_13_rawdata.txt,0.000075,0.000007+0.000000j,P,22
6,10547,ST10547_chamber37_2019_08_22_rawdata.txt,0.000077,0.000006+0.000000j,P,37
7,10268,ST10268_chamber7_2019_08_20_rawdata.txt,0.000093,0.000005+0.000000j,P,7
...,...,...,...,...,...,...
32404,14304,ST14304_chamber13_2021_04_07_rawdata.txt,0.000078,0.000005+0.000000j,P,13
32406,14319,ST14319_chamber15_2021_04_28_rawdata.txt,0.000056,0.000001+0.000000j,P,15
32407,14495,ST14495_chamber4_2022_06_01_rawdata.txt,0.000062,0.000003+0.000000j,P,4
32408,14032,ST14032_chamber1_2020_03_06_rawdata.txt,0.000059,0.000009+0.000000j,P,1


In [ ]:
# Keep one test/rate per file
# Na, don't do this any more. We can do it in the analysis.

# show straws with multiple tests
#straw_df = straw_df.rename_axis(None)
#straw_df[straw_df.straw.duplicated(keep=False)].sort_values(by=['straw'])

# only keep the tests with the smallest leaks
# straw_df = straw_df.groupby('straw', group_keys=False).apply(lambda x: x.loc[x.rate.idxmin()])

# show that we've removed all but the smallest rates
#straw_df[straw_df.straw.duplicated(keep=False)].sort_values(by=['straw']) # throwing error. maybe no duplicates.

#straw_df.sort_values(by=['straw'])

## 2. Panel, batch, and PPG info

In [ ]:
# organized by their panel; includes batch, position number
#straw_list_file = Path("straws_on_panels.csv")
#straw_list_file = Path("2022-04-07_straws_on_panels.csv")
#straw_list_file = Path("2022-04-15_straws_on_panels.csv") # include paper pull grade
straws_on_panels_file = Path("2022-06-16_straws_on_panels.csv") # extends data up to MN244
assert straws_on_panels_file.is_file()

db_df = pd.read_csv(straws_on_panels_file, sep=",")
db_df.columns = db_df.columns.str.replace(" ", "")
db_df = db_df.rename(columns={"id": "straw"})

## Combine data

In [ ]:
# add leak info to every straw that has an associated panel
straws_on_panels_df = db_df.merge(leak_df, on=['straw'], how = 'left')
straws_on_panels_df

In [ ]:
# add panel info to every leak test
all_straws_df = leak_df.merge(db_df, on=['straw'], how = 'left')
all_straws_df

## Make csvs

In [ ]:
straws_on_panels_df.to_csv("straws_on_panels_MN173_MN244.csv")
all_straws_df.to_csv("all_straws_2022-06-13.csv")

In [24]:
leak_df.to_csv("all_passing_leak_tests_2022-06.csv")